In [1]:
from common import *
from pandas.api.types import CategoricalDtype
from numba import njit

from sklearn.preprocessing import QuantileTransformer as QT
from scipy.stats import norm
ppf = norm.ppf

In [2]:
from given import *


# ------------------------------> looking at `W`

In [84]:
#pd.to_pickle((F,P), '/big/data/saves/train_5searchbase.32.pkl')
F,P = pd.read_pickle('/big/data/saves/train_5fixedsince+1.32.pkl')

### Scratcher

In [ ]:
F.index = P.index = F.index.map(lambda x: '{:x}'.format(x))

In [68]:
from sklearn.model_selection import GroupShuffleSplit

ho = P.quarter>=2015.5
ho.name = None
tr, cv = next(GroupShuffleSplit(n_splits=5, test_size=.5, random_state=44).split(F[~ho], P[~ho], groups=P[~ho].quarter))
_dummy = pd.Series(range(len(P)),index=P.index)
#tr, cv = _dummy.isin(tr), _dummy.isin(cv)

In [71]:
F.index[tr]

Index(['b2e8c', 'b2e8d', 'b2e8e', 'b2e8f', 'b2e90', 'b2e91', 'b2e92', 'b2e93',
       'b2e94', 'b2e95',
       ...
       '321a76', '321a77', '321a78', '321a79', '321a7a', '321a7b', '321a7c',
       '321a7d', '321a7e', '321a7f'],
      dtype='object', length=1346646)

In [72]:
guy = GroupShuffleSplit(n_splits=6, test_size=.5, random_state=44).split(F[~ho], P[~ho], groups=P[~ho].quarter)
for tr, cv in guy:
    plist(P.quarter[_dummy.isin(tr)].unique(), end=',\n')
    plist(P.quarter[_dummy.isin(cv)].unique(), end='\n\n')

[2009.0,2009.75,2010.75,2011.5,2012.25,2012.5,2012.75,2013.0,2013.25,2013.75,2014.0,2014.25,2015.0],
[2009.25,2009.5,2010.0,2010.25,2010.5,2011.0,2011.25,2011.75,2012.0,2013.5,2014.5,2014.75,2015.25]

[2009.25,2009.5,2011.0,2011.25,2011.5,2011.75,2012.0,2012.25,2013.0,2013.5,2014.5,2014.75,2015.0],
[2009.0,2009.75,2010.0,2010.25,2010.5,2010.75,2012.5,2012.75,2013.25,2013.75,2014.0,2014.25,2015.25]

[2011.0,2011.75,2012.0,2012.25,2012.75,2013.25,2013.5,2014.0,2014.25,2014.5,2014.75,2015.0,2015.25],
[2009.0,2009.25,2009.5,2009.75,2010.0,2010.25,2010.5,2010.75,2011.25,2011.5,2012.5,2013.0,2013.75]

[2009.5,2010.0,2010.25,2010.5,2011.0,2011.75,2012.25,2012.5,2013.0,2013.5,2014.25,2014.75,2015.25],
[2009.0,2009.25,2009.75,2010.75,2011.25,2011.5,2012.0,2012.75,2013.25,2013.75,2014.0,2014.5,2015.0]

[2009.75,2010.0,2010.25,2010.75,2011.75,2012.0,2012.25,2012.5,2013.0,2013.5,2013.75,2014.5,2015.0],
[2009.0,2009.25,2009.5,2010.5,2011.0,2011.25,2011.5,2012.75,2013.25,2014.0,2014.25,2014.75,2015.

### stock subsets

In [4]:
Mt = M[M.time>=train_start_time]

In [12]:
# # already in given.ipynb
# inUniCount = Mt.groupby('assetCodeId').universe.sum()
# stocksInUni = (inUniCount != 0).pipe(lambda x: x.index[x])
# assert stocksInUni.is_monotonic
# xStocksInUni = set(stocksInUni) #EDITCELL

In [25]:
W = O()
W.y1 = Mt[['time','assetCodeId','oo']].pivot('time','assetCodeId').oo
W.ay1 = Mt[['time','assetCodeId','aoo']].pivot('time','assetCodeId').aoo
W.y10 = Mt[['time','assetCodeId','ooTEN']].pivot('time','assetCodeId').ooTEN
W.ay10 = Mt[['time','assetCodeId','aooTEN']].pivot('time','assetCodeId').aooTEN
W.u = Mt[['time','assetCodeId','universe']].pivot('time','assetCodeId').universe.fillna(0).astype(int)

In [27]:
C = O()
C.y1 = W.y1.corr()
C.ay1 = W.ay1.corr()
C.y10 = W.y10.corr()
C.ay10 = W.ay10.corr()
C.u = W.u.T @ W.u

In [28]:
@njit
def offdiag_items(ix, x, y):
    m, n = x.shape
    assert (m, n) == y.shape
    assert m == n
    assert (m,) == ix.shape
    t = (n**2-n) // 2
    I = np.full(t, -1, dtype=np.int64)
    J = np.full(t, -1, dtype=np.int64)
    X = np.full(t, 0, dtype=x.dtype)
    Y = np.full(t, 0, dtype=y.dtype)
    k = -1
    for i in range(m):
        for j in range(i+1, n):
            k += 1
            I[k] = ix[i]
            J[k] = ix[j]
            X[k] = x[i,j]
            Y[k] = y[i,j]
    return I, J, X, Y

In [29]:
C.v1 = pd.DataFrame(dict(zip([0, 1, 'Corr', 'Unic'], offdiag_items(C.y1.index.values, C.y1.values, C.u.values))))
C.av1 = pd.DataFrame(dict(zip([0, 1, 'Corr', 'Unic'], offdiag_items(C.ay1.index.values, C.ay1.values, C.u.values))))
C.v10 = pd.DataFrame(dict(zip([0, 1, 'Corr', 'Unic'], offdiag_items(C.y10.index.values, C.y10.values, C.u.values))))
C.av10 = pd.DataFrame(dict(zip([0, 1, 'Corr', 'Unic'], offdiag_items(C.ay10.index.values, C.ay10.values, C.u.values))))

In [5]:
#pd.to_pickle((O.py(W),O.py(C)), big_data/'pair/working_2ONEorTEN.pkl')
(W, C) = pd.read_pickle(big_data/'pair/working_2ONEorTEN.pkl'); W, C = O(**W), O(**C)

In [6]:
for c in ['v1','av1','v10','av10']:
    C[c] = C[c][C[c].Unic!=0]
    C[c].sort_values('Corr', ascending=False, inplace=True)

*see some stuff*

In [26]:
C.v[C.av.Unic>=1900].sort_values('Corr', ascending=False).Corr.pipe(lambda x: sum(x>.65))

1523

In [31]:
assetNameIdAssign.series.loc[3741], assetNameIdAssign.series.loc[3759]

('Syneos Health Inc', 'Varonis Systems Inc')

#### making pair feature table

In [7]:
C.av1.head()

,0,1,Corr,Unic
0,0,1,0.145701,0
1,0,2,0.173884,1703
2,0,3,0.245055,2014
3,0,4,0.357883,2014
4,0,5,0.282466,2014


In [12]:
F['time'] = P.time
F[['oo','time','assetCodeId']].head()#.pivot('time','assetCodeId')

MemoryError: 

In [7]:
def make_E_from_F_P(*, F, P):
    # add stuff to F I need
    F['time'] = P.time
    F['universe'] = P.universe.fillna(0) # not removed because need it for generating pair features
    F['assetCodeIdPivot'] = F.assetCodeId.astype(int)
    # the pivot ===============\___
    E = F.pivot('time','assetCodeIdPivot')
    # remove stuff I added to F
    del F['time'], F['universe'], F['assetCodeIdPivot']
    # end block
    
    E.columns = E.columns.swaplevel()
    E = E.sort_index(axis=1)
    E.columns.rename('assetCodeId', level=0, inplace=True)
    return E

In [11]:
#[['assetCodeId','oo1','oo10','oo(15-5)','aoo1','aoo10','aoo(15-5)']]
#E = make_E_from_F_P(F=F, P=P)

assetCodeId                    3774                                            \
                            vp5dd60    vp5dd9        vp60  wVPm_oo1 wVPm_oo10   
time                                                                            
2016-12-14 22:00:00+00:00       NaN       NaN         NaN       NaN       NaN   
2016-12-15 22:00:00+00:00       NaN       NaN         NaN       NaN       NaN   
2016-12-16 22:00:00+00:00       NaN       NaN         NaN       NaN       NaN   
2016-12-19 22:00:00+00:00       NaN       NaN         NaN       NaN       NaN   
2016-12-20 22:00:00+00:00       NaN       NaN         NaN       NaN       NaN   
2016-12-21 22:00:00+00:00       NaN       NaN         NaN       NaN       NaN   
2016-12-22 22:00:00+00:00       NaN       NaN         NaN       NaN       NaN   
2016-12-23 22:00:00+00:00  0.000000  0.000000  32342602.0 -0.004909 -0.009318   
2016-12-27 22:00:00+00:00 -0.034725 -0.034725  31238776.0  0.007027 -0.002738   
2016-12-28 22:00:00+00:00 -0.163688 -0.163688  27459082.0  0.006454  0.006772   
2016-12-29 22:00:00+00:00 -0.189982 -0.189982  26746498.0 -0.016812 -0.010134   
2016-12-30 22:00:00+00:00 -0.171936 -0.171936  27233538.0  0.004810  0.002013   

assetCodeId                                                                \
                          wVPm_oo20  wVPm_oo5 wVPm_oo60 wVPm_volatility10   
time                                                                        
2016-12-14 22:00:00+00:00       NaN       NaN       NaN               NaN   
2016-12-15 22:00:00+00:00       NaN       NaN       NaN               NaN   
2016-12-16 22:00:00+00:00       NaN       NaN       NaN               NaN   
2016-12-19 22:00:00+00:00       NaN       NaN       NaN               NaN   
2016-12-20 22:00:00+00:00       NaN       NaN       NaN               NaN   
2016-12-21 22:00:00+00:00       NaN       NaN       NaN               NaN   
2016-12-22 22:00:00+00:00       NaN       NaN       NaN               NaN   
2016-12-23 22:00:00+00:00  0.023441 -0.005402  0.058461          0.004068   
2016-12-27 22:00:00+00:00  0.030369  0.006015  0.064917          0.004718   
2016-12-28 22:00:00+00:00  0.044178  0.008459  0.068411          0.005010   
2016-12-29 22:00:00+00:00  0.017983 -0.011127  0.050074          0.007266   
2016-12-30 22:00:00+00:00  0.020244 -0.003431  0.056706          0.007122   

assetCodeId                                                                     
                          wVPm_volatility20 wVPm_volatility5 wVPm_volatility60  
time                                                                            
2016-12-14 22:00:00+00:00               NaN              NaN               NaN  
2016-12-15 22:00:00+00:00               NaN              NaN               NaN  
2016-12-16 22:00:00+00:00               NaN              NaN               NaN  
2016-12-19 22:00:00+00:00               NaN              NaN               NaN  
2016-12-20 22:00:00+00:00               NaN              NaN               NaN  
2016-12-21 22:00:00+00:00               NaN              NaN               NaN  
2016-12-22 22:00:00+00:00               NaN              NaN               NaN  
2016-12-23 22:00:00+00:00          0.005844         0.003733          0.007407  
2016-12-27 22:00:00+00:00          0.005974         0.004435          0.007447  
2016-12-28 22:00:00+00:00          0.005701         0.004834          0.007475  
2016-12-29 22:00:00+00:00          0.006804         0.008733          0.007820  
2016-12-30 22:00:00+00:00          0.006850         0.009150          0.007828

In [13]:
#pd.to_pickle(E, big_data/'pair/E_2ONEorTEN.pkl')
E = pd.read_pickle(big_data/'pair/E_2ONEorTEN.pkl')

In [221]:
def make_C_D_L_from_E_Cv(*, E, Cv, cutoff, roll_corr=list(product(['oo','ooTEN'],[10,21,62,250]))):
    Cv = Cv[(Cv.Corr>=cutoff[0])&(Cv.Unic>=cutoff[1])]
    hgt = sum(p[-1] for p in Cv.itertuples())
    Fcolumns = [c for c in E.columns.levels[1] if c!='universe']
    D0, D1, L = np.zeros((hgt,len(Fcolumns))), np.zeros((hgt,len(Fcolumns))), np.zeros((hgt,1+len(roll_corr)))
    #^ -1 on len(Fcolumns) because I take away the 'universe' column
    
    i = 0
    for ii, (_,a0,a1,corr,unic) in enumerate(Cv.itertuples()):
        print_progress(ii)
        A0, A1 = E[a0], E[a1]
        inUniD = (A0.universe.fillna(0)!=0) & (A1.universe.fillna(0)!=0)
        assert inUniD.sum()==unic
        del A0['universe'], A1['universe']
        
        def iter_roll_corr():
            for y, r in roll_corr:
                if a0==1782 and a1==2523 and r==10:
                    BUG.x0 = A0[y]#[inUniD]
                    BUG.x1 = A1[y]#[inUniD]
                yield A0[y].rolling(window=r, min_periods=5).corr(A1[y]) #TODO hard coded min_periods
        B = np.stack(chain(iter_roll_corr(),[np.ones_like(A0.iloc[:,0])*unic]), axis=1)
        
        D0[i:i+unic], D1[i:i+unic] = A0[inUniD].values, A1[inUniD].values
        L[i:i+unic] = B[inUniD]
        
        i += unic
        
    D0 = pd.DataFrame(D0, columns=Fcolumns)
    D1 = pd.DataFrame(D1, columns=Fcolumns)
    L = pd.DataFrame(L, columns=[f'_corr_{y}_{r}' for y,r in roll_corr]+['bothInUniCount'])
    return (D0,D1), L

In [222]:
%%time
D, L = make_C_D_L_from_E_Cv(E=E,Cv=C.av1,cutoff=(.8,500))

0.........10.........20....CPU times: user 1.95 s, sys: 298 ms, total: 2.25 s
Wall time: 2.23 s


#### *still need but later*

In [223]:
def make_C_G_Q_from_D_F_P_W(*, D, F, P, W, cutoff, roll_corr=list(product(['y1','y10'],[10,21,62,250]))):
    allTime = pd.Series(P.time.unique())
    allTime.index = allTime
    
    D = D[(D.Corr>=cutoff[0])&(D.Unic>=cutoff[1])]
    hgt = sum(p[-1] for p in D.itertuples())
    CC = np.zeros((hgt,1+len(roll_corr)))
    G0, G1 = np.zeros((hgt,len(F.columns))), np.zeros((hgt,len(F.columns)))
    Q0, Q1 = [], []
    
    i = 0
    for ii, (_,a0,a1,corr,unic) in enumerate(D.itertuples()):
        print_progress(ii)
        
        in0, in1 = (F.assetCodeId==a0)&(P.universe!=0), (F.assetCodeId==a1)&(P.universe!=0)
        A0, A1 = F[in0], F[in1]
        B0, B1 = P[in0], P[in1]
        inTime = set(B0.time)&set(B1.time)
        tm0, tm1 = B0.time.isin(inTime), B1.time.isin(inTime)
        A0, A1 = A0[tm0], A1[tm1]
        B0, B1 = B0[tm0], B1[tm1]
        
        tm = allTime.isin(inTime).values
        def iter_roll_corr():
            for y, r in roll_corr:
                if a0==1782 and a1==2523 and r==10:
                    BUG.y0 = W[y][a0]#[tm]
                    BUG.y1 = W[y][a1]#[tm]
                yield W[y][a0].rolling(window=r, min_periods=5).corr(W[y][a1])[tm] #TODO hard coded min_periods
        C = np.stack(chain(iter_roll_corr(),[np.ones_like(W[next(iter(W))].iloc[:,0][tm])*unic]), axis=1)
        
        G0[i:i+unic], G1[i:i+unic] = A0.values, A1.values
        Q0.append(B0); Q1.append(B1);
        CC[i:i+unic] = C
        
        i += unic
        
    Q0, Q1 = pd.concat(Q0, axis=0), pd.concat(Q1, axis=0)
    G0, G1 = pd.DataFrame(G0, index=Q0.index, columns=F.columns), pd.DataFrame(G1, index=Q1.index, columns=F.columns)
    C = pd.DataFrame(CC, columns=[f'_corr_{y}_{r}' for y,r in roll_corr]+['bothInUniCount'])
    return C, (G0,G1), (Q0,Q1)

In [224]:
%%time
CC, G, Q = make_C_G_Q_from_D_F_P_W(D=C.av1,F=F,P=P,W=W,cutoff=(.8,500))

0.........10.........20....CPU times: user 5.12 s, sys: 313 ms, total: 5.44 s
Wall time: 5.42 s


In [227]:
guy = pd.DataFrame(np.stack([BUG.x0,BUG.x1,BUG.y0,BUG.y1],axis=1), columns='x0 x1 y0 y1'.split())
guy

,x0,x1,y0,y1
0,0.026586,NaN,0.026586,NaN
1,0.030140,NaN,0.030140,NaN
2,0.156909,NaN,0.156909,NaN
3,0.115705,NaN,0.115705,NaN
4,0.077218,NaN,0.077218,NaN
5,0.133743,NaN,0.133743,NaN
6,0.051907,NaN,0.051907,NaN
7,-0.027886,NaN,-0.027886,NaN
8,-0.055213,NaN,-0.055213,NaN
9,-0.107011,NaN,-0.107011,NaN


In [188]:
CC.columns = L.columns
CC.index = L.index
filt = ((CC-L).abs()>.5).any(axis=1).values

In [189]:
L[filt]

,_corr_oo_10,_corr_oo_21,_corr_oo_62,_corr_oo_250,_corr_ooTEN_10,_corr_ooTEN_21,_corr_ooTEN_62,_corr_ooTEN_250,bothInUniCount
9327,0.777489,0.777489,0.537344,0.927049,0.367031,0.367031,0.705360,0.892839,1108.0
9328,0.697788,0.697788,0.595391,0.925514,0.133736,0.133736,0.635024,0.892534,1108.0
9394,0.817798,0.817798,0.817798,0.817798,0.911308,0.911308,0.911308,0.911308,1108.0
9399,0.819611,0.757520,0.757520,0.757520,-0.148173,0.246991,0.246991,0.246991,1108.0
9400,0.808829,0.761483,0.761483,0.761483,-0.106822,0.249369,0.249369,0.249369,1108.0
9401,0.810037,0.733771,0.733771,0.733771,-0.057490,0.249004,0.249004,0.249004,1108.0
9402,0.760414,0.704362,0.704362,0.704362,-0.102222,0.269724,0.269724,0.269724,1108.0
9403,0.578306,0.718793,0.718793,0.718793,0.217127,0.294508,0.294508,0.294508,1108.0
9405,0.575673,0.716810,0.716810,0.716810,0.296961,0.307801,0.307801,0.307801,1108.0
9406,0.579056,0.676285,0.676285,0.676285,0.057189,0.276118,0.276118,0.276118,1108.0


In [190]:
CC[filt]

,_corr_oo_10,_corr_oo_21,_corr_oo_62,_corr_oo_250,_corr_ooTEN_10,_corr_ooTEN_21,_corr_ooTEN_62,_corr_ooTEN_250,bothInUniCount
9327,0.774560,0.741283,0.815499,0.919904,0.938377,0.938231,0.953277,0.895679,1108.0
9328,0.832444,0.750995,0.814126,0.919326,0.863488,0.944506,0.952290,0.895672,1108.0
9394,0.258400,0.560764,0.665515,0.794220,0.920987,0.804933,0.818779,0.799025,1108.0
9399,0.819611,0.522197,0.666387,0.792432,-0.148173,0.476455,0.787658,0.798462,1108.0
9400,0.808829,0.515150,0.669134,0.793402,-0.106822,0.478308,0.788945,0.797334,1108.0
9401,0.810037,0.412096,0.665200,0.792128,-0.057490,0.478464,0.792666,0.796569,1108.0
9402,0.760414,0.401210,0.662236,0.790231,-0.102222,0.499328,0.800450,0.797652,1108.0
9403,0.578306,0.420680,0.668541,0.787456,0.217127,0.496399,0.803853,0.795906,1108.0
9405,0.575673,0.362732,0.667244,0.787534,0.296961,0.349843,0.809633,0.797078,1108.0
9406,0.579056,0.676285,0.644377,0.778837,0.057189,0.276118,0.808856,0.794463,1108.0


In [176]:
Q[0][filt]

,time,assetCode,assetName,universe,returnsOpenNextMktres10,quarter,y,target,upDown,absVal,weight,vp1_weight,vp5_weight,vp10_weight,always_weight
2441036,2013-03-15 22:00:00+00:00,BBL.N,BHP Billiton PLC,1.0,-0.046024,2013.00,-0.046024,False,-1,0.046024,0.046024,0.002307,0.002318,0.001831,0.000000
2442750,2013-03-18 22:00:00+00:00,BBL.N,BHP Billiton PLC,1.0,-0.068006,2013.00,-0.068006,False,-1,0.068006,0.068006,0.002470,0.003155,0.002785,0.000000
3049887,2014-08-29 22:00:00+00:00,BBL.N,BHP Billiton PLC,1.0,-0.050749,2014.50,-0.050749,False,-1,0.050749,0.050749,0.001187,0.001485,0.002695,0.000000
3057842,2014-09-08 22:00:00+00:00,BBL.N,BHP Billiton PLC,1.0,-0.059585,2014.50,-0.059585,False,-1,0.059585,0.059585,0.001638,0.002059,0.001901,0.000000
3059434,2014-09-09 22:00:00+00:00,BBL.N,BHP Billiton PLC,1.0,-0.032017,2014.50,-0.032017,False,-1,0.032017,0.032017,0.001687,0.001224,0.001119,0.000000
3061025,2014-09-10 22:00:00+00:00,BBL.N,BHP Billiton PLC,1.0,-0.042329,2014.50,-0.042329,False,-1,0.042329,0.042329,0.001041,0.001620,0.001465,0.000000
3062614,2014-09-11 22:00:00+00:00,BBL.N,BHP Billiton PLC,1.0,-0.065031,2014.50,-0.065031,False,-1,0.065031,0.065031,0.002221,0.002197,0.002303,0.000000
3064204,2014-09-12 22:00:00+00:00,BBL.N,BHP Billiton PLC,1.0,-0.071725,2014.50,-0.071725,False,-1,0.071725,0.071725,0.001435,0.002280,0.002350,0.000000
3067382,2014-09-16 22:00:00+00:00,BBL.N,BHP Billiton PLC,1.0,-0.056554,2014.50,-0.056554,False,-1,0.056554,0.056554,0.002705,0.001646,0.001904,0.000000
3068972,2014-09-17 22:00:00+00:00,BBL.N,BHP Billiton PLC,1.0,-0.044641,2014.50,-0.044641,False,-1,0.044641,0.044641,0.001203,0.001321,0.001515,0.000000


In [205]:
D[1][filt][['assetCodeId','ooTEN']]

,assetCodeId,ooTEN
9327,2523.0,0.006927
9328,2523.0,0.024541
9394,2523.0,-0.064492
9399,2523.0,-0.073686
9400,2523.0,-0.090031
9401,2523.0,-0.058687
9402,2523.0,-0.062219
9403,2523.0,-0.017520
9405,2523.0,-0.034093
9406,2523.0,-0.009496


# ------------------------------> looking at `N`